In [1]:
import requests,json
import time
import pandas as pd
from lxml import etree
import mysql.connector
from multiprocessing import Pool as ThreadPool
import socket 
socket.setdefaulttimeout(10)

In [2]:
def creat_MysqlDB(): 
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="123456",
      database="MOOC"
    )
    return mydb

In [3]:
head = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}

In [143]:
def getsource(url):
    try:
        html = requests.get(url,headers=head,timeout=(3,7))
    except requests.exceptions.ConnectTimeout as e:
        print(e)
        return
    selector = etree.HTML(html.text)
    content = selector.xpath("//html")[0]
    course_id = int(url.replace('http://www.imooc.com/learn/', ''))
    course_name = content.xpath('//*[@id="main"]/div[1]/div[1]/div[2]/h2/text()')
    if course_name:
        course_name = course_name[0] #课程名称
        course_direction = content.xpath('//*[@id="main"]/div[1]/div[1]/div[1]/a[2]/text()')[0] #课程方向
        course_category = content.xpath('//*[@id="main"]/div[1]/div[1]/div[1]/a[3]/text()')[0] #课程类别
        course_level = content.xpath('//*[@id="main"]/div[1]/div[1]/div[3]/div[2]/span[2]/text()')[0] #课程难度
        course_duration = content.xpath('//*[@id="main"]/div[1]/div[1]/div[3]/div[3]/span[2]/text()')[0] #课程时长
        number_learning_url = 'http://www.imooc.com/course/AjaxCourseMembers?ids='+ str(course_id) #学习人数json
        try:
            number_learning = int(json.loads(requests.get(number_learning_url).text)['data'][0]['numbers']) #解析学习人数
        except requests.exceptions.ConnectTimeout as e:
            print(e)
            return
        course_score = float(content.xpath('//*[@id="main"]/div[1]/div[1]/div[3]/div[5]/span[2]/text()')[0])#评分

        #问答数量
        number_QnA = content.xpath('//*[@id="main"]/div[2]/div/ul/li[2]/span/text()') 
        if number_QnA:
            number_QnA = int(number_QnA[0])
        else:
            number_QnA = 0
        #评论数量
        number_comment = content.xpath('//*[@id="main"]/div[2]/div/ul/li[4]/span/text()') 
        if number_comment:
            number_comment = int(number_comment[0])
        else:
            number_comment = 0

        #计算笔记数量
        #1.无课程笔记 2.不需要翻页 3.需要翻页    
        note_url = 'http://www.imooc.com/course/note/id/' + str(course_id)
        note_html = requests.get(note_url,headers=head,timeout=(3,7))
        note_selector = etree.HTML(note_html.text)
        note_content = note_selector.xpath("//html")[0]
        first_page_note_number = len(note_content.xpath('//*[@id="js-note-container"]/li'))
        last_page_note = note_content.xpath('//*[@id="course_note"]/div/a[last()]/@href')
        if last_page_note:
            last_page_note = int(last_page_note[0].split('=')[-1])
            note_number = first_page_note_number + (last_page_note-1)*15
        else:
            note_number = first_page_note_number
        course_chapter = len(content.xpath('//*[@class="course-chapters"]/div'))#视频章节
        code_number = len(content.xpath('//i[@class="imv2-code_circle type"]')) #代码练习数量
        video_number = len(content.xpath('//i[@class="imv2-play_circle type"]'))#视频数量
        exercise_number = len(content.xpath('//i[@class="imv2-exercise_circle type"]'))#测评数量
        sql = "INSERT INTO courses_list (course_id,course_name,course_direction,course_category,course_level\
                    ,course_duration,number_learning\
                    ,course_score,number_QnA,number_comment,note_number\
                    ,course_chapter,code_number,video_number,exercise_number)\
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)"
        val = (course_id,course_name,course_direction,course_category,course_level
               ,course_duration,number_learning
               ,course_score,number_QnA,number_comment,note_number
               ,course_chapter,code_number,video_number,exercise_number)
        mydb = creat_MysqlDB()
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)
        mydb.commit()
    
    
    

In [145]:
def spider():
    pool = ThreadPool(8) 
    for i in range(6,1104):
        url = 'http://www.imooc.com/learn/' + str(i)
        try:
            pool.apply_async(getsource, (url,))
        except Exception as e:
            print(e)
            continue
    pool.close()
    pool.join()  

In [146]:
spider()

In [25]:
len([])

0